In [1]:
# load and show dataset
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../../Dataset/UNFCCC.xlsx')
data.head()

,Country_code,Country,Format_name,Pollutant_name,Sector_code,Sector_name,Parent_sector_code,Unit,Year,emissions,Notation,PublicationDate,DataSource
0,BE,Belgium,IPCC Common Reporting Format,CH4,2.C.7,2.C.7 - Other Metal Industry,2.C,Gg,1997,0.0,NO,20190528.0,EEA
1,BE,Belgium,IPCC Common Reporting Format,CH4,2.D,2.D - Non-energy Products from Fuels and Solve...,2,Gg,2011,0.0,"NO,NA",20190528.0,EEA
2,BE,Belgium,IPCC Common Reporting Format,CH4,2.D,2.D - Non-energy Products from Fuels and Solve...,2,Gg,2012,0.0,"NO,NA",20190528.0,EEA
3,BE,Belgium,IPCC Common Reporting Format,CH4,2.D.1,2.D.1 - Lubricant Use,2.D,Gg,2013,0.0,NaN,20190528.0,EEA
4,BE,Belgium,IPCC Common Reporting Format,CH4,2.D.1,2.D.1 - Lubricant Use,2.D,Gg,2017,0.0,NaN,20190528.0,EEA


In [5]:
# Column Names spaces subbed with '_'
column_names = ["Year", "Country", \
                # do not change these two
                "Energy_Emissions", "Industrial_Process_Emissions",\
                "Agriculture_Emissions", "Land_Use_Emissions", "Transport_Emissions",\
                "Total CO2 Emissions"]
sector_codes = ["1","2","3","4","1.A.3","Sectors/Totals_excl_incl_avi"]
dataMain = pd.DataFrame(columns = column_names)
dataMain.head()

,Year,Country,Energy_Emissions,Industrial_Process_Emissions,Agriculture_Emissions,Land_Use_Emissions,Transport_Emissions,Total CO2 Emissions


In [37]:
column_names[1]

'Country'

In [13]:
countries = pd.Series(data['Country']).dropna().unique()
print(countries)
years = pd.Series(data['Year']).dropna().unique()
print(years)
# Hungary has data for 1985-1987 logged in as a single row for each contributor

['Belgium' 'Austria' 'Bulgaria' 'Switzerland' 'Cyprus' 'Czechia' 'Germany'
 'Denmark' 'Estonia' 'Spain' 'EU28 (Convention)' 'EU (KP)' 'Finland'
 'France' 'Greece' 'Croatia' 'Hungary' 'Ireland' 'Iceland' 'Italy'
 'Liechtenstein' 'Lithuania' 'Luxembourg' 'Latvia' 'Malta' 'Netherlands'
 'Norway' 'Poland' 'Portugal' 'Romania' 'Sweden' 'Slovenia' 'Slovakia'
 'Turkey' 'United Kingdom (Convention)']
['1997' '2011' '2012' '2013' '2017' '1990' '1996' '2000' '2007' '2008'
 '2009' '1999' '2006' '1998' '2003' '1991' '1995' '2001' '2005' '2010'
 '2015' '1993' '2014' '1994' '2016' '2002' '1992' '2004' '1989' '1988'
 '1985-1987' '1987' '1985' '1986']


In [14]:
for rowIter in data.iterrows():
    row = rowIter[1]
    if(str(row['Pollutant_name']) == 'CO2'):
        exist_index = list((dataMain['Country'].astype(str) == str(row['Country'])) & (dataMain['Year'].astype(str) == str(row['Year'])))
        exist_row = dataMain.iloc[exist_index]
        if(exist_row.empty == True ):
            new_row = pd.DataFrame(columns = column_names)
            new_row.at[0,'Country'] = row['Country']
            new_row.at[0,'Year'] = row['Year']
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    new_row.at[0,column_names[i+2]] = row['emissions']
            dataMain = pd.concat([dataMain,new_row],ignore_index=True)
        else:
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    dataMain.at[exist_index,column_names[i+2]] = row['emissions']

In [15]:
dataMain.head()

,Year,Country,Energy_Emissions,Industrial_Process_Emissions,Agriculture_Emissions,Land_Use_Emissions,Transport_Emissions,Total CO2 Emissions
0,2005,Belgium,103904,20906,164.694,-1609.97,26347.5,129089
1,1995,Belgium,105789,19941.6,178.345,-2658.36,22519.5,128989
2,2010,Belgium,96868.7,16092.5,171.411,-432.823,26168.1,117996
3,2015,Belgium,83906.3,15374.6,180.773,-446.97,26404.1,104148
4,1990,Belgium,101544,18459.5,178.47,-3325.1,20553.8,123607


In [16]:
dataMain['Energy_Emissions'] = dataMain['Energy_Emissions'] - dataMain['Transport_Emissions']

In [17]:
dataMain.head()

,Year,Country,Energy_Emissions,Industrial_Process_Emissions,Agriculture_Emissions,Land_Use_Emissions,Transport_Emissions,Total CO2 Emissions
0,2005,Belgium,77556.8,20906,164.694,-1609.97,26347.5,129089
1,1995,Belgium,83269.5,19941.6,178.345,-2658.36,22519.5,128989
2,2010,Belgium,70700.5,16092.5,171.411,-432.823,26168.1,117996
3,2015,Belgium,57502.2,15374.6,180.773,-446.97,26404.1,104148
4,1990,Belgium,80990.7,18459.5,178.47,-3325.1,20553.8,123607


In [18]:
dataMain.to_excel('../../Dataset/DataMain.xlsx', index=False)

In [10]:
# Creating ENERGY EMISSION SUB-MODEL data
column_names = ["Year", "Country", \
                # do not change these two
                "Public_Electricity_and_Heat_Production","Petroleum_Refining",\
                "Manufacturing_Industries","International_Aviation",\
                "International_Navigation","Energy_CO2_Emissions","Transport_Emissions"]
sector_codes = ["1.A.1.a","1.A.1.b","1.A.2","1.D.1.a","1.D.1.b","1","1.A.3"]
data_energy = pd.DataFrame(columns = column_names)
data_energy.head()

,Year,Country,Public_Electricity_and_Heat_Production,Petroleum_Refining,Manufacturing_Industries,International_Aviation,International_Navigation,Energy_CO2_Emissions,Transport_Emissions


In [14]:


for rowIter in data.iterrows():
    row = rowIter[1]
    if(str(row['Pollutant_name']) == 'CO2'):
        exist_index = list((data_energy['Country'].astype(str) == str(row['Country'])) & 
                           (data_energy['Year'].astype(str) == str(row['Year'])))
        exist_row = data_energy.iloc[exist_index]
        if(exist_row.empty == True ):
            new_row = pd.DataFrame(columns = column_names)
            new_row.at[0,'Country'] = row['Country']
            new_row.at[0,'Year'] = row['Year']
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    new_row.at[0,column_names[i+2]] = row['emissions']
            data_energy = pd.concat([data_energy,new_row],ignore_index=True)
        else:
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    data_energy.at[exist_index,column_names[i+2]] = row['emissions']

data_energy['Energy_CO2_Emissions'] = data_energy['Energy_CO2_Emissions']-data_energy['Transport_Emissions'];
data_energy = data_energy.drop(['Transport_Emissions'],axis=1)
data_energy.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Country,Energy_CO2_Emissions,International_Aviation,International_Navigation,Manufacturing_Industries,Petroleum_Refining,Public_Electricity_and_Heat_Production,Year
0,Belgium,77556.8,3569.15,25219.3,18359.1,4353.23,24473,2005
1,Belgium,83269.5,2908.39,12968.6,22735.2,4088.89,24182.4,1995
2,Belgium,70700.5,4182.35,24882.1,15399.6,4709.69,21371.4,2010
3,Belgium,57502.2,4395.13,18872.6,13319.2,4681.84,16214.7,2015
4,Belgium,80990.7,3125.53,13312.5,23084.8,4299.17,23536.6,1990


,Year,Country,Public_Electricity_and_Heat_Production,Petroleum_Refining,Manufacturing_Industries,International_Aviation,International_Navigation,Energy_CO2_Emissions


In [16]:
data_energy.to_excel('../../Dataset/DataEnergyEmissions.xlsx', index=False)

In [ ]:
# Creating TRANSPORT EMISSION SUB-MODEL data
# Copy and change column names and put sector_codes from the main UNFCCC.xlsx file
column_names = ["Year", "Country", \
                # do not change these two
                "Domestic_Aviation","Road_Transportation",\
                "Railways","Transport_Emissions"]
sector_codes = ["1.A.3.a","1.A.3.b","1.A.3.c","1.A.3"]
data_transport = pd.DataFrame(columns = column_names)

In [153]:
for rowIter in data.iterrows():
    row = rowIter[1]
    if(str(row['Pollutant_name']) == 'CO2'):
        exist_index = list((data_transport['Country'].astype(str) == str(row['Country'])) 
                           & (data_transport['Year'].astype(str) == str(row['Year'])))
        exist_row = data_transport.iloc[exist_index]
        if(exist_row.empty == True ):
            new_row = pd.DataFrame(columns = column_names)
            new_row.at[0,'Country'] = row['Country']
            new_row.at[0,'Year'] = row['Year']
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    new_row.at[0,column_names[i+2]] = row['emissions']
            data_transport = pd.concat([dataMain,new_row],ignore_index=True)
        else:
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    dataMain.at[exist_index,column_names[i+2]] = row['emissions']
data_transport.head()

In [ ]:
data_transport.to_excel('../../Dataset/DataTransportEmissions.xlsx', index=False)

In [154]:
# Creating INDUSTRIAL PROCESSES EMISSION SUB-MODEL		 data
# Copy and change column names and put sector_codes from the main UNFCCC.xlsx file
column_names = ["Year", "Country", \
                # do not change these two
                "Mineral_Industry","Chemical_Industry",\
                "Metal_Industry","Industrial_Process_Emissions"]
sector_codes = ["2.A","2.B","2.C","2"]
dataMain = pd.DataFrame(columns = column_names)

for rowIter in data.iterrows():
    row = rowIter[1]
    if(str(row['Pollutant_name']) == 'CO2'):
        exist_index = list((dataMain['Country'].astype(str) == str(row['Country'])) & (dataMain['Year'].astype(str) == str(row['Year'])))
        exist_row = dataMain.iloc[exist_index]
        if(exist_row.empty == True ):
            new_row = pd.DataFrame(columns = column_names)
            new_row.at[0,'Country'] = row['Country']
            new_row.at[0,'Year'] = row['Year']
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    new_row.at[0,column_names[i+2]] = row['emissions']
            dataMain = pd.concat([dataMain,new_row],ignore_index=True)
        else:
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    dataMain.at[exist_index,column_names[i+2]] = row['emissions']
dataMain.head()

In [ ]:
data_transport.to_excel('../../Dataset/DataIndusProcessEmissions.xlsx', index=False)

In [155]:
# Creating LAND USE EMISSION SUB-MODEL data
# Copy and change column names and put sector_codes from the main UNFCCC.xlsx file
column_names = ["Year", "Country", \
                # do not change these two
                "Forestland","Cropland ","Grassland ",\
                "Harvested_Wood_Products","Land_Use_Emissions"]
sector_codes = ["4.A","4.B","4.C","4.G","4"]
dataMain = pd.DataFrame(columns = column_names)

for rowIter in data.iterrows():
    row = rowIter[1]
    if(str(row['Pollutant_name']) == 'CO2'):
        exist_index = list((dataMain['Country'].astype(str) == str(row['Country'])) & (dataMain['Year'].astype(str) == str(row['Year'])))
        exist_row = dataMain.iloc[exist_index]
        if(exist_row.empty == True ):
            new_row = pd.DataFrame(columns = column_names)
            new_row.at[0,'Country'] = row['Country']
            new_row.at[0,'Year'] = row['Year']
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    new_row.at[0,column_names[i+2]] = row['emissions']
            dataMain = pd.concat([dataMain,new_row],ignore_index=True)
        else:
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    dataMain.at[exist_index,column_names[i+2]] = row['emissions']
dataMain.head()
dataMain.to_excel("Dataset/DataLandUseEmissions.xlsx")

In [156]:
# Creating International Bunkers EMISSION SUB-MODEL data
# Copy and change column names and put sector_codes from the main UNFCCC.xlsx file
# Change file name
column_names = ["Year", "Country", \
                # do not change these two
                "International_Aviation","International_Navigation ",\
                "International_Bunkers_Emissions"]
sector_codes = ["1.D.1.a","1.D.1.b","1.D.1"]
dataMain = pd.DataFrame(columns = column_names)

for rowIter in data.iterrows():
    row = rowIter[1]
    if(str(row['Pollutant_name']) == 'CO2'):
        exist_index = list((dataMain['Country'].astype(str) == str(row['Country'])) & (dataMain['Year'].astype(str) == str(row['Year'])))
        exist_row = dataMain.iloc[exist_index]
        if(exist_row.empty == True ):
            new_row = pd.DataFrame(columns = column_names)
            new_row.at[0,'Country'] = row['Country']
            new_row.at[0,'Year'] = row['Year']
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    new_row.at[0,column_names[i+2]] = row['emissions']
            dataMain = pd.concat([dataMain,new_row],ignore_index=True)
        else:
            for i in range(0,len(sector_codes)):
                if(row['Sector_code'] == sector_codes[i]):
                    dataMain.at[exist_index,column_names[i+2]] = row['emissions']
dataMain.head()
dataMain.to_excel("Dataset/DataIntBunkersEmissions.xlsx")